In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
    
pd.set_option("display.max_columns", None)

data_intermediate_dir = "data/intermediate/"

item_lvl_w_imp = pd.read_csv(data_intermediate_dir + "item_lvl_w_impairment.csv")
[print(x) for x in item_lvl_w_imp.columns]

Basic_Demos,Age
Basic_Demos,Sex
WIAT,WIAT_Num_Stnd
WIAT,WIAT_Word_Stnd
SympChck,CSC_01C
SympChck,CSC_01P
SympChck,CSC_02C
SympChck,CSC_02P
SympChck,CSC_03C
SympChck,CSC_03P
SympChck,CSC_04C
SympChck,CSC_04P
SympChck,CSC_05C
SympChck,CSC_05P
SympChck,CSC_06C
SympChck,CSC_06P
SympChck,CSC_07C
SympChck,CSC_07P
SympChck,CSC_08C
SympChck,CSC_08P
SympChck,CSC_09C
SympChck,CSC_09P
SympChck,CSC_10C
SympChck,CSC_10P
SympChck,CSC_11C
SympChck,CSC_11P
SympChck,CSC_12C
SympChck,CSC_12P
SympChck,CSC_13C
SympChck,CSC_13P
SympChck,CSC_14C
SympChck,CSC_14P
SympChck,CSC_15C
SympChck,CSC_15P
SympChck,CSC_16C
SympChck,CSC_16P
SympChck,CSC_17C
SympChck,CSC_17P
SympChck,CSC_18C
SympChck,CSC_18P
SympChck,CSC_19C
SympChck,CSC_19P
SympChck,CSC_20C
SympChck,CSC_20P
SympChck,CSC_21C
SympChck,CSC_21P
SympChck,CSC_22C
SympChck,CSC_22P
SympChck,CSC_23C
SympChck,CSC_23P
SympChck,CSC_24C
SympChck,CSC_24P
SympChck,CSC_25C
SympChck,CSC_25P
SympChck,CSC_26C
SympChck,CSC_26P
SympChck,CSC_27C
SympChck,CSC_27P
SympChck,CS

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
diag_distrib = data_up_to_SCARED[["Diag: " +x for x in diags]].sum().sort_values(ascending=False)

## Predict consensus diagnosis
### Specific Learning Disorder with Impairment in Reading

In [26]:
diag_cols = [x for x in item_lvl_w_imp.columns if x.startswith("Diag: ")]

chosen_diag_cols = [
    "Diag: Specific Learning Disorder with Impairment in Reading",
    "Diag: Specific Learning Disorder with Impairment in Mathematics",
    "Diag: Intellectual Disability-Mild",
    "Diag: Borderline Intellectual Functioning",
    "Diag: Specific Learning Disorder with Impairment in Written Expression",
    "Diag: ADHD-Combined Type", #(very common non-learning)
    "Diag: Disruptive Mood Dysregulation Disorder" #(non-common non-learning)
]

for diag_col in chosen_diag_cols:
    if item_lvl_w_imp[diag_col].sum() > 10:
        output_col = diag_col

        input_cols = [x for x in item_lvl_w_imp.columns if 
                       not x.startswith("Diag: ") 
                       and not x.startswith("WIAT")
                       and not x.startswith("WISC")]

        X_train, X_test, y_train, y_test = train_test_split(item_lvl_w_imp[input_cols], item_lvl_w_imp[output_col], test_size=0.3, random_state=0)

        from sklearn.model_selection import StratifiedKFold
        from sklearn.model_selection import cross_val_score
        from sklearn.linear_model import LogisticRegression

        from sklearn import svm

        from sklearn.model_selection import cross_val_score
        svc = svm.SVC(kernel='rbf', C=1, random_state=42)
        lr = svm.SVC(kernel='linear', C=1, random_state=42)
        cv = StratifiedKFold(n_splits=3)
        svc_auc = cross_val_score(svc, X_train, y_train, cv=cv, scoring="roc_auc")
        lr_auc = cross_val_score(lr, X_train, y_train, cv=cv, scoring="roc_auc")
        svc_recall = cross_val_score(svc, X_train, y_train, cv=cv, scoring="recall")
        lr_recall = cross_val_score(lr, X_train, y_train, cv=cv, scoring="recall")
        svc_precision = cross_val_score(svc, X_train, y_train, cv=cv, scoring="precision")
        lr_precision = cross_val_score(lr, X_train, y_train, cv=cv, scoring="precision")

        print(output_col)
        print(svc_auc.mean(), lr_auc.mean(), svc_recall.mean(), svc_recall.mean(), svc_precision.mean(), svc_precision.mean())

/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Diag: Specific Learning Disorder with Impairment in Reading
0.5841117666338905 0.5891203703703706 0.0 0.0 0.0 0.0


/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Diag: Specific Learning Disorder with Impairment in Mathematics
0.6212083694441005 0.6162973051050308 0.0 0.0 0.0 0.0


/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: Undefi

...

/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Diag: Specific Learning Disorder with Impairment in Written Expression
0.5101141230023031 0.5498479246372844 0.0 0.0 0.0 0.0
Diag: ADHD-Combined Type
0.8331869497937389 0.7841304916695316 0.43198171337706226 0.43198171337706226 0.6915538132573058 0.6915538132573058


/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: Undefi

Diag: Disruptive Mood Dysregulation Disorder
0.885783916976924 0.7880862965161244 0.0 0.0 0.0 0.0


/Users/home/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
for train_index, test_index in cv.split(X_train, y_train):
    print("Positive classes in train: ", y_train.iloc[train_index].sum())
    print("Positive classes in test: ", y_train.iloc[test_index].sum())

Positive classes in train:  9
Positive classes in test:  4
Positive classes in train:  9
Positive classes in test:  4
Positive classes in train:  8
Positive classes in test:  5
